# Análisis de los experimentos

> En este notebook se pretenden analizar los resultados de los experimentos propuestos


In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns

import os
import shutil
import random, math

## Funciones útiles

In [17]:
# Creación de una carpeta

def crear_carpeta(dir):
    if os.path.exists(dir) : shutil.rmtree(dir)
    os.makedirs(dir)

In [18]:
# Armamos el horno basandonos en el experimento del que se trata, su valor n y su valor m

def armar_horno(exp,m,n):
    valores = []
    
    with open(F"resultados/EXP{exp}/D{exp}_{m}_{n}-resultado.out", "r") as f:
        for linea in f:
            valores.append(float(linea))
    
    horno = np.array(valores).reshape(m,n).T
    return horno

In [19]:
# Dado las temperaturas de un horno, calcula las posiciones en los radios en los que 
# se encuentra la isoterma buscada.
def calcular_isoterma(horno):
    # Datos
    isoterma = 500
    pos_isoterma = []
    n = len(horno)
    m = len(horno[0])
    temp_ext = horno[0][0]
    temp_int = horno[0][-1]
    # Si la isoterma buscada es mayor que la temperatura exterior/menor que la interior
    # consideramos que no hay isoterma
    if (temp_int > temp_ext and (isoterma > temp_int or temp_ext > isoterma)):
        pos_isoterma.append(-1)
    elif (temp_int < temp_ext and (isoterma < temp_int or temp_ext < isoterma)):
        pos_isoterma.append(-1)
    else:
        for i in range(0,n):
            radio_j = -1
            diferencia = abs(horno[0][0] - horno[0][m-1])
            for j in range(0,m):
                if (abs(isoterma - horno[i][j] < diferencia)):
                    diferencia = abs(isoterma - horno[i][j])
                    radio_j = j
            pos_isoterma.append(radio_j)       
    return pos_isoterma

In [20]:
# Calculamos la diferencia entre las isotermas de hornos 
def calcular_diferencias(isotermas_n, r_i, r_e, ms):
    
    d_rs = []
    
    for i in range(len(ms)):
        d_rs.append((r_e - r_i)/ms[i])
    
    normas = []
    diferencias = []
    
    for j in range(len(isotermas_n)):
        radios = []
        for i in range(40):
            radios.append(r_i + isotermas_n[j][i]* d_rs[j])
        norma = np.linalg.norm(radios)
        normas.append(norma)
    
    for i in range(len(normas)):
        diferencia = abs(normas[i-1]-normas[i])
        diferencias.append(diferencia)
        
    return diferencias

## Funciones para graficar

In [21]:
# Grafica la isoterma de un horno
def grafico_isoterma(exp,horno,m,n):
    # Datos y parámetros
    cantidad_radios   = m
    cantidad_angulos  = n
    r = calcular_isoterma(horno)
    theta = np.linspace(0, 2*np.pi, cantidad_angulos)
    # Gráfico
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.plot(theta, r)
    ax.set_rticks([x*0.1*m for x in range(1,11)])
    ax.grid(True)
    ax.set_title("Isoterma")
    #plt.show()

In [22]:
# Arma el gráfico de calor para un horno 
def grafico_calor(exp, horno):
    # Datos y parámetros
    cantidad_angulos = len(horno)
    cantidad_radios  = len(horno[0])
    r = np.tile(np.linspace(0, 1, cantidad_radios), (cantidad_angulos, 1))
    theta = np.tile(np.linspace(0, 2*np.pi, cantidad_angulos), (cantidad_radios, 1)).transpose()
    z = horno
    # Gráfico
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    grafico = ax.pcolor(theta, r, z, cmap='jet')
    ax.pcolor(theta, r, z, cmap='jet')
    ax.set_yticklabels([])
    fig.colorbar(grafico)
    plt.savefig(F"graficos/EXP{exp}/D{exp}_{cantidad_radios}_{cantidad_angulos}_calor.svg")

## Preparación

In [38]:
resultados_d1 = pd.read_csv("resultados/EXP1/resultado.csv")
resultados_d2 = pd.read_csv("resultados/EXP2/resultado.csv")

In [39]:
crear_carpeta("graficos/EXP1")
crear_carpeta("graficos/EXP2")
crear_carpeta("graficos/EXP3A")
crear_carpeta("graficos/EXP3B")

## Análisis de Experimento 1

In [ ]:
# Tomamos una serie de hornos, y realizamos su gráfico de calor
d_m = 2
n = 40
m = 40
hornos = []

for j in range(4,m+1,d_m):
    hornos.append(armar_horno(1,j,n))
    
for i in range(len(hornos)):
    grafico_calor(1,hornos[i])

In [ ]:
# Analizamos las diferencias de isoterma con los hornos de n=40
d_m = 2
m_i = 4
m = 40
n = 40
hornos = []

for j in range(4,m+1,d_m):
    hornos.append(armar_horno(1,j,n))
    
isotermas = []   
for horno in hornos:
    isotermas.append(calcular_isoterma(horno))
    
ms = []
for horno in hornos:
    ms.append(len(horno[0]))
    
diferencias = calcular_diferencias(isotermas, resultados_d1['Radio interno'], resultados_d1['Radio externo'], ms)


print(diferencias)

In [ ]:
ys = [i for i in range(4,41,2)]
plt.plot(ys, diferencias)
plt.xlabel("Valor de m (radios)")
plt.ylabel("Diferencia en cm")
plt.grid(True)
plt.savefig(F"graficos/EXP1/grafico_diferencias.png")

## Análisis de Experimento 2


In [52]:
 D2_tiempos = resultados_d2.copy()

In [ ]:
#Gráfico Gauss y LU
met_dic = ['Gauss']*30 + ['LU']*30
fig, ax = plt.subplots()

ax.errorbar(data=D2_tiempos[D2_tiempos['Método'] == 0], x='Número de instancia', y='Media', yerr='Desvio', marker ="o",capsize=3, capthick=1, alpha=.75, label="Gauss", linestyle="none")
ax.errorbar(data=D2_tiempos[D2_tiempos['Método'] == 1], x='Número de instancia', y='Media', yerr='Desvio', marker ="o",capsize=3, capthick=1, alpha=.75, label="LU", linestyle="none")


ax.set(xlabel="Número de Instancia del Horno", ylabel="Tiempo (ms)")
ax.grid(True)
plt.legend()
plt.savefig(F"graficos/EXP2/tiempos_gauss_lu.png")

In [ ]:
#Gráfico solo Gauss
fig, ax = plt.subplots()

ax.errorbar(data=D2_tiempos[D2_tiempos['Método'] == 0], x='Número de instancia', y='Media', yerr='Desvio', marker ="o",capsize=3, capthick=1, alpha=.75, linestyle="none")

ax.set(xlabel="Número de Instancia del Horno", ylabel="Tiempo (ms)")
ax.grid(True)

plt.savefig(F"graficos/EXP2/tiempos_gauss.png")

In [ ]:
#Gráfico de tiempo de LU
fig, ax = plt.subplots()

ax.errorbar(data=D2_tiempos[D2_tiempos['Método'] == 1], x='Número de instancia', y='Media', yerr='Desvio', marker ="o",capsize=3, capthick=1, alpha=.75, linestyle="none")

ax.set(xlabel="Número de Instancia del Horno", ylabel="Tiempo (ms)")
ax.grid(True)

plt.savefig(F"graficos/EXP2/tiempos_lu.png")

In [ ]:
#Gráfico Acercamiento LU
fig, ax = plt.subplots()

ax.errorbar(data=D2_tiempos[(resultados_d2['Método'] == 1) & (resultados_d2['Número de instancia'] != 1)], x='Número de instancia', y='Media', yerr='Desvio', marker ="o",capsize=3, capthick=1, alpha=.75, linestyle="none")

ax.set(xlabel="Número de Instancia del Horno", ylabel="Tiempo (ms)")
ax.grid(True)

plt.savefig(F"graficos/EXP2/tiempos_lu_acercado.png")

## Análisis de Experimento 3A

In [51]:
# Dado un archivo con resultados, calcula el horno que está desde la posición i hasta j
def conseguir_n_horno(array,i,j):
    horno = []
    for k in range(i,j):
        horno.append(array[k])
    return horno    

In [12]:
# Armado de los hornos
resultado = []

with open(F"resultados/EXP3A/D3_30_40-resultado.out", "r") as f:
    for linea in f:
        resultado.append(float(linea))
        
ninst = len(resultado)//(40*30)

hornos = []    
for k in range(0,ninst):
    horno= []
    horno = conseguir_n_horno(resultado,30*40*k, 30*40*(k+1))
    hornos.append(np.array(horno).reshape(30,40).T)

In [ ]:
def grafico_calor_d3(exp, horno, k):
    # Datos y parámetros
    cantidad_angulos = len(horno)
    cantidad_radios  = len(horno[0])
    r = np.tile(np.linspace(0, 1, cantidad_radios), (cantidad_angulos, 1))
    r[-1] = r[0]
    theta = np.tile(np.linspace(0, 2*np.pi, cantidad_angulos), (cantidad_radios, 1)).transpose()
    z = horno
    # Gráfico
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    grafico = ax.pcolor(theta, r, z, cmap='jet')
    ax.pcolor(theta, r, z, cmap='jet')
    ax.set_yticklabels([])
    fig.colorbar(grafico)
    plt.savefig(F"graficos/EXP{exp}A/D3_{cantidad_radios}_{cantidad_angulos}_{k}_calor.png")


for i in range(0,len(hornos)):
    grafico_calor_d3(3, hornos[i],i)

In [ ]:
# Grafica la isoterma de un horno
def grafico_isoterma_d3(exp,horno,k):
    # Datos y parámetros
    cantidad_angulos = len(horno)
    cantidad_radios  = len(horno[0])
    r = calcular_isoterma(horno)
    r[-1] = r[0]
    theta = np.linspace(0, 2*np.pi, cantidad_angulos)
    # Gráfico
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.plot(theta, r)
    ax.set_rticks([x*0.1*cantidad_radios for x in range(1,11)])
    ax.grid(True)
    plt.savefig(F"graficos/EXP{exp}A/D3_{cantidad_radios}_{cantidad_angulos}_{k}_isoterma.png")

for i in range(0,len(hornos)):
    grafico_isoterma_d3(3, hornos[i],i)

## Análisis de Experimento 3B

In [ ]:
# Armado de los hornos
resultado = []

with open(F"resultados/EXP3B/D4_30_40-resultado.out", "r") as f:
    for linea in f:
        resultado.append(float(linea))
        
ninst = len(resultado)//(40*30)

hornos = []    
for k in range(0,ninst):
    horno= []
    horno = conseguir_n_horno(resultado,30*40*k, 30*40*(k+1))
    hornos.append(np.array(horno).reshape(30,40).T)

In [ ]:
def grafico_calor_d3(exp, horno, k):
    # Datos y parámetros
    cantidad_angulos = len(horno)
    cantidad_radios  = len(horno[0])
    r = np.tile(np.linspace(0, 1, cantidad_radios), (cantidad_angulos, 1))
    r[-1] = r[0]
    theta = np.tile(np.linspace(0, 2*np.pi, cantidad_angulos), (cantidad_radios, 1)).transpose()
    z = horno
    # Gráfico
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    grafico = ax.pcolor(theta, r, z, cmap='jet')
    ax.pcolor(theta, r, z, cmap='jet')
    ax.set_yticklabels([])
    fig.colorbar(grafico)
    plt.savefig(F"graficos/EXP{exp}B/D4_{cantidad_radios}_{cantidad_angulos}_{k}_calor.png")


for i in range(0,len(hornos)):
    grafico_calor_d3(3, hornos[i],i)

In [ ]:
def isoterma_s_fallados(horno):
    # Datos
    isoterma = 1000
    pos_isoterma = []
    n = len(horno)
    m = len(horno[0])
    temp_ext = horno[0][0]
    temp_int = horno[0][-1]
    for i in range(0,n):
        radio_j = -1
        diferencia = abs(horno[0][0] - horno[0][m-1])
        for j in range(0,m):
            if (abs(isoterma - horno[i][j] < diferencia)):
                diferencia = abs(isoterma - horno[i][j])
                radio_j = j
        pos_isoterma.append(radio_j)       
    return pos_isoterma

In [ ]:
# Grafica la isoterma de un horno
def grafico_isoterma_d3(exp,horno,k):
    # Datos y parámetros
    cantidad_angulos = len(horno)
    cantidad_radios  = len(horno[0])
    r = isoterma_s_fallados(horno)
    r[-1] = r[0]
    theta = np.linspace(0, 2*np.pi, cantidad_angulos)
    # Gráfico
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.plot(theta, r)
    ax.set_rticks([x*0.1*cantidad_radios for x in range(1,11)])
    ax.grid(True)
    plt.savefig(F"graficos/EXP{exp}B/D4_{cantidad_radios}_{cantidad_angulos}_{k}_isoterma.png")

for i in range(0,len(hornos)):
    grafico_isoterma_d3(3, hornos[i],i)